# Chapter 3: Exercise 13

**Import packages**

In [2]:
import numpy as np
# import pandas as pd
# import statsmodels.api as sm
# import matplotlib.pyplot as plt
# from ISLP.models import (ModelSpec as MS, summarize)

### (a) Generate a vector x containing n=100 observations from N(0, 1) distribution

In [20]:
rng = np.random.default_rng(1)
x = rng.normal(size=100)

### (b) Generate a vector eps containing n=100 observations from N(0, 0.25) (mean=0, variance=0.25)

In [53]:
eps = rng.normal(loc=0, scale=np.sqrt(0.25), size=100)

### (c) Using x and eps generate a vector y according to $Y = -1 + 0.5X + \epsilon$

In [58]:
y = -1 + 0.5 * x + eps

In [59]:
len(y)

100

- **length(y)** = 100
- **$\beta_0$** = -1
- **$\beta_1$** = 0.5